In [3]:
import tensorflow as tf
import os

class MyFashionMnist(object):
  def train(self):

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.summary()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=10)

    model.evaluate(x_test,  y_test, verbose=2)

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='fairing-job',
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-gpu',
            registry=DOCKER_REGISTRY, 
            push=True)
        # cpu 2, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='dudaji',
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=2,
                                                                       memory=5)]
         
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 200215 14:17:36 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7fced5104828>
[I 200215 14:17:36 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fcec7267128>
[I 200215 14:17:36 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fcec7267128>
[W 200215 14:17:36 append:50] Building image using Append builder...
[I 200215 14:17:36 base:105] Creating docker context: /tmp/fairing_context_pnv40848
[I 200215 14:17:36 converted_notebook:127] Converting fashion-mnist-fairing.ipynb to fashion-mnist-fairing.py
[I 200215 14:17:36 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-gpu'
[W 200215 14:17:38 append:54] Image successfully built in 2.131834232946858s.
[W 200215 14:17:38 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/fairing-job:4D2DC88F...
[I 2002

4423680/4422102 [==============================] - 0s 0us/step
2020-02-15 14:17:44.600484: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-02-15 14:17:44.648306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-15 14:17:44.649457: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties:
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-02-15 14:17:44.649549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-15 14:17:44.650682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties:
name: Tesla K80 major: 3 minor: 7 

[W 200215 14:19:03 job:162] Cleaning up job fairing-job-82gd8...
